In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt

IMG_DIR = '/run/media/hennes/T7/projects/Seminar Paper/data/images/ae1ff67f-4e46-4eea-a00f-9c3e08de0760/composite.tif'


# Open the .tif file
with rasterio.open(IMG_DIR) as src:
    # Read the bands (assuming band 1 is infrared, band 2 is red, band 3 is green, band 4 is blue)
    blue = src.read(1)
    green = src.read(2)
    red = src.read(3)
    infrared = src.read(4)

    # Stack the bands into a single array
    img = np.stack((infrared, red, green), axis=-1)

    # Normalize the image data to 0-1 range for display
    img_min, img_max = img.min(), img.max()
    img = (img - img_min) / (img_max - img_min)

    # Display the image
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()


In [12]:
from datetime import datetime
import pandas as pd

import getpass
from planet import Auth, Session
from planet import order_request
from planet_helpers import repair_aoi, get_offline_order_names

import asyncio
import nest_asyncio
nest_asyncio.apply()

from download_images import prepare_data, create_order_list
from tqdm.asyncio import tqdm  # Use tqdm.asyncio for async support

# initialise logging
import logging
logging.basicConfig(
    filename='../download_images.log',  # Log file location
    level=logging.ERROR,  # Set the logging level
    format='%(asctime)s - %(levelname)s - %(message)s'  # Log message format
)

DATA_DIR = "../data"
DOWNLOAD_DIR = DATA_DIR + "/images"
SEARCH_DIR = DATA_DIR + "/searches"
ORDER_DIR = DATA_DIR + "/orders"

In [18]:
sample_df = pd.read_csv(DATA_DIR + "/download_sample.csv")
places_df = pd.read_csv(DATA_DIR + "/places.csv")[["location_id", "geometry"]]
sample = prepare_data(sample_df, places_df)    
order_list = create_order_list(sample)

In [33]:
search = [order for order in order_list if "events_9173475" in order["name"]][0]
geom = search["geom"]["coordinates"][0]

In [ ]:
from shapely.geometry import Polygon
from shapely.validation import make_valid
polygon = Polygon(geom)
if not polygon.is_valid:
    repaired_geom = make_valid(polygon)
    repaired_geom = repaired_geom.geoms[0].exterior.coords[:]
    repaired_geom = [list(pair) for pair in repaired_geom]
    return repaired_geom
else:
    return geom